In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\user\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\user\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\traitlets

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\user\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\user\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\traitlets

AttributeError: _ARRAY_API not found

In [20]:
input_file_path = 'C:/Users/user/TRAINEE/superstore_bi_pipeline/data/raw/initial_load.csv'

output_file_path = 'C:/Users/user/TRAINEE/superstore_bi_pipeline/data/raw/secondary_load_2024.csv'

In [3]:
new_records_size = 300

In [10]:
start_row_id = 10895
rows_count = 2500  # Number of orders

Подготовка справочников

In [6]:
# We read the source code if it is not already in memory
try:
    df_ref = pd.read_csv(input_file_path)
except:
   # If the file is not found, use the df variable from memory (if available))
    df_ref = df.copy()

In [7]:
# Unique clients (we take them with addresses)
customers_db = df_ref[[
    'customer_id', 'customer_name', 'segment', 
    'country', 'city', 'state', 'postal_code', 'region'
]].drop_duplicates(subset=['customer_id'])

In [8]:
# Unique products
products_db = df_ref[[
    'product_id', 'category', 'sub_category', 'product_name'
]].drop_duplicates(subset=['product_id'])

In [24]:
# Shipping Methods
ship_modes = df_ref['ship_mode'].unique()

print(f"User ID: {start_row_id}")
print(f"Number of lines: {rows_count}")

User ID: 10895
Number of lines: 2500


3. Generation

In [12]:
data = []
start_date_2024 = datetime(2024, 1, 1)

In [13]:
for i in range(rows_count):
    # Date in 2024
    order_date = start_date_2024 + timedelta(days=random.randint(0, 364))
    ship_date = order_date + timedelta(days=random.randint(1, 5))
    
    # Random customer and product
    cust = customers_db.sample(1).iloc[0]
    prod = products_db.sample(1).iloc[0]
    
    # Metrics
    qty = random.randint(1, 9)
    price = random.uniform(10, 400)
    discount = random.choice([0, 0, 0, 0.1, 0.2])
    sales = round(price * qty * (1 - discount), 2)
    profit = round(sales * random.uniform(-0.15, 0.35), 2)
    
    row = {
        'row_id': start_row_id + i,
        # Unique Order ID with the GAP prefix
        'order_id': f'GAP-2024-{10000 + i}',
        'order_date': order_date.strftime('%Y-%m-%d'),
        'ship_date': ship_date.strftime('%Y-%m-%d'),
        'ship_mode': random.choice(ship_modes),
        
        # Client
        'customer_id': cust['customer_id'],
        'customer_name': cust['customer_name'],
        'segment': cust['segment'],
        'country': cust['country'],
        'city': cust['city'],
        'state': cust['state'],
        'postal_code': cust['postal_code'],
        'region': cust['region'],
        
        # Product
        'product_id': prod['product_id'],
        'category': prod['category'],
        'sub_category': prod['sub_category'],
        'product_name': prod['product_name'],
        
        # Numbers
        'sales': sales,
        'quantity': qty,
        'discount': discount,
        'profit': profit,
        
        # Technical fields for ETL
        'valid_from': order_date.strftime('%Y-%m-%d'),
        'valid_to': '',
        'previous_region': '',
        'is_new_version': False, # It's just a story, not a change
        'scd2_attribute_changed': ''
    }
    data.append(row)


4. Creating a DataFrame

In [14]:
gap_df = pd.DataFrame(data)

In [15]:
# Fixed data types
gap_df['postal_code'] = pd.to_numeric(gap_df['postal_code'], errors='coerce').fillna(0).astype(int)

In [22]:
#5. Saving to CSV
gap_df.to_csv(output_file_path, index=False, encoding='utf-8')

In [23]:
gap_df.head(3)

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,...,product_name,sales,quantity,discount,profit,valid_from,valid_to,previous_region,is_new_version,scd2_attribute_changed
0,10895,GAP-2024-10000,2024-02-28,2024-03-03,First Class,AG-10765,Anthony Garverick,Home Office,United States,Philadelphia,...,"Executive Impressions 16-1/2"" Circular Wall Clock",297.76,9,0.2,-42.10,2024-02-28,,,False,
1,10896,GAP-2024-10001,2024-02-02,2024-02-06,Same Day,AH-10210,Alan Hwang,Consumer,United States,Brentwood,...,Microsoft Natural Keyboard Elite,651.64,7,0.0,128.23,2024-02-02,,,False,
2,10897,GAP-2024-10002,2024-02-12,2024-02-16,First Class,CA-11965,Carol Adams,Corporate,United States,Hoover,...,Computer Printout Paper with Letter-Trim Perfo...,114.53,4,0.2,-3.77,2024-02-12,,,False,
